<div style="border:solid green 2px; padding: 20px">
<b>Юрий, привет!</b>

Меня зовут Евгений Д., и я буду проверять твой проект. Предлагаю общаться на «ты» 😊 Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать аналитиком данных. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - пожалуйста, не перемещай, не изменяй и не удаляй их. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера:</b> Так выделены небольшие замечания. Одна-два таких замечания в проекте допустимы, но если их будет больше - тебе будет необходимо внести исправления. Это как тестовое задание при приеме на работу: очень много мелких ошибок могут стать причиной отказа кандидату. 

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом. Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Вступление в работу очень важно. Так ты поясняешь то, чему она посвящена. Можно также привести краткий план проведения самой работы и описание используемых столбцов. 

</div>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd #импорт Pandas
data = pd.read_csv('/datasets/data.csv') #чтение CSV файла
data.info() #получение информации о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

В данных присутствуют артефакты. Например, отрицательное количество дней трудового стажа. Необходимо разобраться в них и исправить.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#ищем пропущенные значение используя метод isnull()
print (data.isnull().sum())



children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


### Вывод

После использования метода для поиска пропущенных значений и изучения результатов его работы, можно сделать вывод, что пропущенные значения присутствуют в столбцах 'days_employed' и 'total_income'

In [4]:
def null_to_mean (income_type_list,column):
    #функция принимает на вход DataFrame
    #список типов занятости, для которых нужно
    #провести расчет минимального уровня дохода.
    #Затем для каждого типа занятости подставляет
    #соответствующий минимальный доход вместо NaN
    for income_source in income_type_list:
        mean_column = data[data.income_type == income_source][column].mean()
        data.loc[data.income_type == income_source] =\
        data.loc[data.income_type == income_source].fillna(mean_column)
    print(f'Пропущенных значений в столбце {column}:{data[column].isna().sum()}')
    return data

#Проверяем результат
data = null_to_mean(data.income_type.unique(), 'total_income')
data = null_to_mean(data.income_type.unique(), 'days_employed')

Пропущенных значений в столбце total_income:0
Пропущенных значений в столбце days_employed:0


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Классная функция 👍
    
Особенно радует, что она используется без дополнительного метода `apply` 😊 
    
Т.к. на больших данных метод `apply` будет работать очень долго. 
    
</div>

### Замена типа данных

In [5]:
data['days_employed'] = data['days_employed'].astype('int64') #заменяем значения на с float на int
data['total_income'] = data['total_income'].astype('int64') #заменяем значения с float на int
data['family_status'] = data['family_status'].str.lower() #заменяем строки в Series с заглавными буквам на маленькие


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

В том числе мы можем менять тип сразу у нескольких столбцов одновременно с помощью метода `astype` 

</div>

### Вывод

Мы заменили float на int через метод .astype(). Метод int() и метод to_numeric здесь не могли пройти т.к нам нужно было перевести именно float в int. Так-же заменили некоторые строки все строки на нижний регистр
Теперь необходимо разобраться с дубликатами.

### Обработка дубликатов

In [6]:
data.duplicated().sum() #ищем общее кол-во дубликатов в таблице
data = data.drop_duplicates().reset_index(drop = True) #сбрасываем дубликаты и заменяем индексы сброшенные дубликатов.



### Вывод

Сначала мы нашли все дубликаты в таблице используя метод .duplicated().sum(). Далее мы сбросили их и заменили их индекс используя drop_duplicates().reset_index(drop = True). Дубликаты могли появиться из-за неправильной дублирования строк с одинаковой информацией или похожими по смыслу, но разными по названию категориями.

<div class="alert alert-danger">
<h2> Комментарий ревьюера</h2>

<s>Обрати внимание, что таким способом мы не удалили дубликаты, так мы не переопределили переменную. </s>

</div>


In [7]:
print(f'Количество дубликатов: {data.duplicated().sum()}')

Количество дубликатов: 0


<div class="alert alert-info"> <b>Комментарий студента:</b> Присвоил сброс дубликатов data. Теперь показывает 0 дубликатов.</div>

### Лемматизация

In [8]:
phrases = data['purpose'].unique() #находим уникальные значения столбца purpose
from pymystem3 import Mystem #импорт библиотеки
lemmas_list = [] #сохраняем пустой список для лемматизированных слов
m = Mystem() #сохраняем в переменной функцию
for phrase in phrases: #пишем цикл, лемматизирующий уникальные значения и добавляем их в список с лемматизированными словами.
    lemmas = ''.join(m.lemmatize(phrase)).strip()
    lemmas_list.append(lemmas)
print(lemmas_list)

def to_category(row):#пишем функцию, которая будет проверять слова из каждого ряда purpose и собирать их в отдельную категорию
        purpose = m.lemmatize(row['purpose'])
        if 'жилье' in purpose:
            return 'операции с жильем'
        if ('коммерческий' in purpose) or ('жилой' in purpose ):
            return 'недвижимость'
        if 'автомобиль' in purpose:
            return 'операции с автомобилем'
        if 'образование' in purpose:
            return 'получение образования'
        if 'свадьба' in purpose:
            return 'кредит на проведение свадьбы'
data['purpose'] = data.apply(to_category, axis=1)
print(data['purpose'].value_counts())
print(data['purpose'].isna().sum())


['покупка жилье', 'приобретение автомобиль', 'дополнительный образование', 'сыграть свадьба', 'операция с жилье', 'образование', 'на проведение свадьба', 'покупка жилье для семья', 'покупка недвижимость', 'покупка коммерческий недвижимость', 'покупка жилой недвижимость', 'строительство собственный недвижимость', 'недвижимость', 'строительство недвижимость', 'на покупка подержать автомобиль', 'на покупка свой автомобиль', 'операция с коммерческий недвижимость', 'строительство жилой недвижимость', 'жилье', 'операция со свой недвижимость', 'автомобиль', 'заниматься образование', 'сделка с подержанный автомобиль', 'получение образование', 'автомобиль', 'свадьба', 'получение дополнительный образование', 'покупка свой жилье', 'операция с недвижимость', 'получение высокий образование', 'свой автомобиль', 'сделка с автомобиль', 'профильный образование', 'высокий образование', 'покупка жилье для сдача', 'на покупка автомобиль', 'ремонт жилье', 'заниматься высокий образование']
операции с жильем

<div class="alert alert-danger">
<h2> Комментарий ревьюера</h2>

<s>Поправь пожалуйста код, чтобы я смог проверить твой проект дальше. </s>
    
</div>
<div class="alert alert-info"> <b>Комментарий студента:</b> Поправил код, и дописал немного.</div>

### Вывод

Мы лемматизировали данные из столбца purpose. Для этого была использована библиотека pymystem3. Для начала был составлен список лемматизированных значений. Потом была составлена функция, которые эти значения проверяет, по каждой выделенной группе. Затем мы заменили эти значения уже в столбце purpose, на группы на которые разбили их. Далее подсчитали их и сделали вывод, что кредиты на операции с жильем, берут больше всего. На втором месте идут операции с автомобилем, а на третем месте получение образования.

### Категоризация данных

In [9]:

#Разбивает людей по группам исходя из их доходов, используя правила:
#'малоимущие' при значении total_income <= 20000
#'средняя зп' при значении total_income < 50000
#'зп выше среднего' при значении total_income >= 50000

def total_income_categorize(row):
    total_income = row['total_income']


    if total_income <= 20000:
        return 'малоимущие'

    if total_income < 50000:
        return 'средняя зп'
    
    if total_income >= 50000:
        return 'зп выше среднего'

#data.apply(total_income_categorize, axis=1)
#print(data.head(10)) 
data['total_income'] = data.apply(total_income_categorize, axis=1)
print(data['total_income'].value_counts()) 

#total_income_categorize()
            

зп выше среднего    21099
средняя зп            372
Name: total_income, dtype: int64


### Вывод

Мы написали функцию и разбили людей на группы по их доходам, затем подсчитали кол-во людей в каждой группе. Людей с зарплатой выше 50000 рублей мы отнесли к группе с зарплатой выше среднего, а от 20 до 50 - со средней зарплатой. Людей с зарплатой выше среднего оказалось гораздо больше, целых в 56 раз. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [23]:

pivot_table_income_level = data.pivot_table(index='children', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_income_level[1]/(pivot_table_income_level[1] + pivot_table_income_level[0]) * 100


children
-1      2.127660
 0      7.535266
 1      9.232689
 2      9.454191
 3      8.181818
 4      9.756098
 5           NaN
 20    10.526316
dtype: float64

### Вывод

Исходя из полученного результата, можно сделать вывод, что доля невозврата кредита самая высокая у семьи с 20 детьми. В целом она растет по количеству детей, хотя у семьи с 3 детьми доля ниже, чем у тех где их 4, либо 2.

<div class="alert alert-danger">
<h2> Комментарий ревьюера</h2>

<s>Обрати внимание, что мы не совсем верно посчитали для ответа на вопросы.
    
Здесь нам нужно посчитать вероятность клиента стать должником.
    
Как она расчитывается ? 
    
Вероятность = кол–во клиентов не вернувших долг / общее кол–во клиентов в группе
    
Это посчитать можно с помощью сводных таблиц `pivot_table`, хочу напомнить, что сводные таблицы это те же датафремы. То есть в них можно добавлять столбцы и делать нужные нам расчеты.
    
Давай поправим пожалуйста эти расчеты. </s>
    
</div>


<div class="alert alert-info"> <b>Комментарий студента:</b> Исправил код, сделал через pivot. Сначала находим необходимую нам завимисоть в сводной таблице, затем долю клиентов, которые не отдали кредит в зависимости от вопроса.</div>


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Верно 👍
    
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:

pivot_table_income_level = data.pivot_table(index='family_status', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_income_level[1]/(pivot_table_income_level[1] + pivot_table_income_level[0]) * 100


family_status
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.320202
женат / замужем          7.542126
не женат / не замужем    9.750890
dtype: float64

### Вывод

Исходя из полученного результата, доля у людей состоящих в гражданском браке или не состоящих в браке выше, чем у остальных.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В точку 👍
    
Обрати внимание, что наши действия повторяются с точки зрения расчетов.
    
Это говорит о том, что мы можем создать функцию для постоянных расчетов. Тем самым мы сможем оптимизировать свой код 😊
    
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:

pivot_table_income_level = data.pivot_table(index='total_income', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_income_level[1]/(pivot_table_income_level[1] + pivot_table_income_level[0]) * 100


total_income
зп выше среднего    8.142566
средняя зп          6.182796
dtype: float64

### Вывод

Исходя из полученных данных люди с зарплатой выше среднего имеют большую долю.


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Верно 👍
    

    
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
pivot_table_income_level = data.pivot_table(index='purpose', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_income_level[1]/(pivot_table_income_level[1] + pivot_table_income_level[0]) * 100


purpose
кредит на проведение свадьбы    7.965739
недвижимость                    7.392843
операции с автомобилем          9.354689
операции с жильем               6.904282
получение образования           9.217738
dtype: float64

### Вывод

Исходя из полученного результата, можно сделать вывод, что люди берущие кредит на операции с автомобилем и на получение образования, имеют большую долю, чем остальные.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Верно 👍
    
</div>

### Шаг 4. Общий вывод

Кредиты бывают разные и на скорость их возврата играет много факторов. Каждый случай уникален и нельзя сказать в целом, что все люди берущие кредиты на свадьбу имеют задолжность, хотя в некоторых случаях да. Для правильного рассмотрение, необходимо анализировать человека по всем параметрам и смотреть на долю клиентов из каждого среза, которая не отдала кредит.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Выводы понятны и логичны, а самое главное подкреплены выявленными фактами.
       
В целом сам проект выполнен на хорошем уровне! На протяжении всей работы чувствуется глубина проработки задачи и это безусловный плюс для нас. Было использовано большое количество методов, которые будут помогать тебе в дальнейших проектах.
    
Не стоит забывать про оформление своей работы, это важный такой же важный этап. Рекомендую «не стесняться» использовать ячейки типа **markdown** там где нам необходимо отобразить свои размышления, в том числе с использованием дополнительной стилизации.
    
Успехов тебе в новых проектах! 
    
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Рекомендую посмотреть на дополнительную стилизацию ячеек типа **markdown**.
    
**Жирный**
*Курсив*
    
--- 

# Заголовок первого уровня

--- 
    
## Заголовок второго уровня
  
---  
Списки:
    
- один 
- два
- три
   
---
    
1. Раз
2. Раз раз
3. Раз два три
    
--- 
    
Отображение `переменных`
    
    
---
   
Также здесь можно найти все, что нужно:
    
https://www.notion.so/35d9c2f32ffa42f3b17010737aedca4b
    
</div>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.